This code is for Example 3 on domain $(-1,1) \times (-1,1)$ when $A = -100(cos(f), sin(f)))$, where $f = \pi *\sin(\pi*x)*\cos(\pi*y)$

# This section is to display the graph of A and F

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
# Generate computation domain (-1,1) x (-1,1)
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
# Generate a triangular mesh of mesh-size 0.01, modify maxh value for resutls with h=0.03 and h=0.05
mesh = Mesh(geo.GenerateMesh(maxh=0.01))
# Define vector field A and FEM space for solving the Neumann problem, fix the local polynomial degree at p=3
a =100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

# Define trial-function, test-function, and the variation form 
u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

# the solution field
gfm.vec.data = a.mat.Inverse() * f.vec

F = A - grad(gfu)
F0x, F1x = F.Diff(x)
F0y, F1y = F.Diff(y)
Draw(A, mesh,vector=True)
Draw(F, mesh,vector=True)

Calculate the $L^2$ norm for A and F

In [20]:
sqrt(Integrate(A*A, mesh)), sqrt(Integrate(F*F, mesh))

(199.99999999999977, 118.96129807129384)

# This section is to calculate the eigenpairs for H(A)

In [21]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))

domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))

X = H1(mesh, order=3, complex=True, dirichlet = '.*')
# X.ndof
psi, phi = X.TnT()
a3 = BilinearForm(X)
a3 += grad(psi) * grad(phi) * dx +  1j * InnerProduct(A,grad(phi)) * psi * dx - 1j * InnerProduct(A , grad(psi)) * phi * dx \
       + InnerProduct(A,A)* psi * phi *dx
b3 = BilinearForm(X)
b3 += psi * phi * dx
a3.Assemble()
b3.Assemble()

# Use FEAST algorithm for computing eigenvalues
seed =1
npts=10
nspan=6
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object, change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 21, center = 115; When h=0.03, radius = 22, center = 136; When h=0.05, radius = 26, center = 332
P = SpectralProjNG(X,
                   a3.mat,
                   b3.mat,
                   radius=26,
                   center=332,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(X, nspan, M=b3.mat)
Y.setrandom(seed=seed)
start2 = time.time()
print('Prepare time', start2-start1)
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Total Time", end - start1, "FEAST Time", end - start2)
y = Y.gridfun()
Draw(y)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=26, Center=332+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +356.727 +8.034j
SpectralProjNG:   Factorizing at z = +347.282+21.034j
SpectralProjNG:   Factorizing at z = +332.000+26.000j
SpectralProjNG:   Factorizing at z = +316.718+21.034j
SpectralProjNG:   Factorizing at z = +307.273 +8.034j
SpectralProjNG:   Factorizing at z = +307.273 -8.034j
SpectralProjNG:   Factorizing at z = +316.718-21.034j
SpectralProjNG:   Factorizing at z = +332.000-26.000j
SpectralProjNG:   Factorizing at z = +347.282-21.034j
SpectralProjNG:   Factorizing at z = +356.727 -8.034j
Prepare time 0.8570871353149414

=========== Starting FEAST iterations ===========
Trying with 6 vectors:

 ITERATION 1 with 6 vectors
   Real part of computed eigenvalues:
   [313.80226668 324.03033835 327.02977534 344.77687537 352.16142512
 358.83439886]
   Relative Hausdorff distance fr

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the degree of freedom 

In [23]:
X.ndof 

17017

Calculate the $L^2$ norm of $\nabla \psi$ and $A\psi$

In [24]:
for i in range(0,6):
    psi = GridFunction(X)
    psi.Set(y.MDComponent(i))
    print(sqrt(Integrate(grad(psi)*Conj(grad(psi)), mesh)), sqrt(Integrate(A*psi*Conj(A*psi), mesh)))

(95.01347227647042+0j) (99.99999999999999+0j)
(95.5517671150651+0j) (99.99999999999973+0j)
(95.43300286819841+0j) (99.99999999999969+0j)
(94.93397414867844+0j) (99.99999999999991+0j)
(95.46892009419035+0j) (99.99999999999983+0j)
(94.91370830404492+0j) (99.99999999999996+0j)


# This section is to calculate the eigenpairs for H(F)

In [1]:
import time
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from pyeigfeast import NGvecs, SpectralProjNG
start1 = time.time()
domain = WorkPlane().MoveTo(-1,-1).Rectangle(2,2).Face()
geo = OCCGeometry(domain, dim = 2)
mesh = Mesh(geo.GenerateMesh(maxh=0.05))
a = 100
A = CF((-a*cos(pi*sin(pi*x)*cos(pi*y)), -a*sin(pi*sin(pi*x)*cos(pi*y))))
X = H1(mesh, order=3)
N = NumberSpace(mesh)
fesm = X*N

u, lam = fesm.TrialFunction()
v, c = fesm.TestFunction()
gfm = GridFunction(fesm)  # solution
gfu, gflam = gfm.components

a = BilinearForm(fesm)
a += lam*v*dx + grad(u)*grad(v)*dx + u*c*dx
a.Assemble()
f = LinearForm(A*grad(v)*dx).Assemble()

gfm.vec.data = a.mat.Inverse() * f.vec
F = A - grad(gfu)

Xeig = H1(mesh, order=3, complex = True, dirichlet = '.*')
psi, phi = Xeig.TnT()
a2 = BilinearForm(Xeig)
a2 += grad(psi) * grad(phi) * dx + 1j *  InnerProduct(F,grad(phi)) * psi * dx - 1j * InnerProduct(F , grad(psi)) * phi * dx\
    +  InnerProduct(F, F)* psi * phi *dx
b2 = BilinearForm(Xeig)

b2 += psi * phi * dx
a2.Assemble()
b2.Assemble()

seed =1
npts=10
nspan=6
within=None
rhoinv=0.0
quadrule='circ_trapez_shift'
verbose=True
# Make the spectral projector object,  change radius and center values for the searching range of eigenvalues.
# When h=0.01, radius = 21, center = 115; When h=0.03, radius = 21, center = 115; When h=0.05, radius = 25, center = 121
P = SpectralProjNG(Xeig,
                   a2.mat,
                   b2.mat,
                   radius=25,
                   center=121,
                   npts=npts,
                   within=within,
                   rhoinv=rhoinv,
                   quadrule=quadrule,
                   inverse=None)
Y = NGvecs(Xeig, nspan, M=b2.mat)
Y.setrandom(seed=seed)
start2 = time.time()
lam, Y, history, _ = P.feast(Y)
end = time.time()
print("Time", end - start2)

y1 = Y.gridfun()
Draw(y1)


SpectralProj: Setting shifted trapezoidal rule quadrature on circular contour
SpectralProj: Radius=25, Center=121+0j

SpectralProjNG: Computing resolvents using umfpack
SpectralProjNG:   Factorizing at z = +144.776 +7.725j
SpectralProjNG:   Factorizing at z = +135.695+20.225j
SpectralProjNG:   Factorizing at z = +121.000+25.000j
SpectralProjNG:   Factorizing at z = +106.305+20.225j
SpectralProjNG:   Factorizing at z = +97.224 +7.725j
SpectralProjNG:   Factorizing at z = +97.224 -7.725j
SpectralProjNG:   Factorizing at z = +106.305-20.225j
SpectralProjNG:   Factorizing at z = +121.000-25.000j
SpectralProjNG:   Factorizing at z = +135.695-20.225j
SpectralProjNG:   Factorizing at z = +144.776 -7.725j

=========== Starting FEAST iterations ===========
Trying with 6 vectors:

 ITERATION 1 with 6 vectors
   Real part of computed eigenvalues:
   [ 96.68848472 122.13203821 129.16838555 129.21427298 144.56684602
 144.72439579]
   Relative Hausdorff distance from prior iterate: 8.264e+97

 ITER

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

Calculate the $L^2$ norm of $\nabla \phi$ and $F\phi$

In [2]:
for i in range(0,6):
    phi = GridFunction(Xeig)
    phi.Set(y1.MDComponent(i))
    print(sqrt(Integrate(grad(phi)*Conj(grad(phi)), mesh)), sqrt(Integrate(F*phi*Conj(F*phi), mesh)))

(36.863380802123984+0j) (36.47789605721692+0j)
(39.624381308055135+0j) (39.00525974634234+0j)
(45.50453894340125+0j) (45.045556701595785+0j)
(45.53365395459239+0j) (45.096345000885336+0j)
(58.70288781061806+0j) (58.74262334778236+0j)
(58.67904772989794+0j) (58.72178757609474+0j)
